In [ ]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten
import random
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re

In [ ]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB','Class']
# dataset = dataset.loc[:,cols]
# d = {'Class': dataset['Class'].value_counts().index, 'count': dataset['Class'].value_counts()}
# df_Class = pd.DataFrame(data=d).reset_index(drop=True)
# df_Class
dataset.sample(10)

In [ ]:
dataset = shuffle(dataset)

In [ ]:
def FReIndex(colIndex,reIndex):
    dirindex = {}
    lisindex = []
    for i in range(len(colIndex)):
        lisindex.append(i)
    dirindex = dict(zip(lisindex,colIndex))
    colIndex = []
    for i in range(len(reIndex)):
        for key in dirindex:
            if int(key) == int(reIndex[i]):
                colIndex.append(dirindex[key])
                break
    print(colIndex)
    return colIndex
    

In [ ]:
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
# lisindex = []
# for i in range(len(dataset.columns)):
#     lisindex.append(i)
# dataset = dataset.reindex(lisindex)
colIndex = ['Area','Minor_Axis', 'Eccentricity',
            'Eqdiasq','Convex_Area', 'Aspect_Ratio',
            'Compactness', 'Shapefactor_1', 
            'Shapefactor_3']

lis = [] #优化算法参数入口（编码）
for i in range(len(colIndex)):
    lis.append(len(colIndex)-i)
    
#colIndex = FReIndex(colIndex,lis)
dataset = pd.DataFrame(dataset,columns=colIndex)
dataset


In [ ]:
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [ ]:

import os,sys
import shutil

shutil.rmtree('models')  
os.mkdir('models')

In [ ]:
#定义模型

sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)

lstm_layers = [1,2,3,4,5]
dense_layers = [1,2,3,4,5,6]
units = [16,32,64,128]
dropout = [0.05,0.1,0.15,0.25]
Batch_size = [32,64,128]
optimizer = ['adam',sgd]
for the_batch_size in Batch_size:
    for the_dropout in dropout:
        for the_optimizer in optimizer:
            for the_dense_layers in dense_layers:
                for the_lstm_layers in lstm_layers:
                    for the_units in units:
                        sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
                        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=random.randint(10,100))
                        X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
                        X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
                        model = Sequential()
                        # model.build(input_shape=(277,277,2))
                        #print(model.summary())
                        #model.add(SpatialDropout1D(0.2))
                        model.add(LSTM(the_units ,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences = True))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        # #第二层
                        for i in range(the_lstm_layers):
                            model.add(LSTM(the_units,return_sequences=True))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())

                        model.add(LSTM(the_units))
                        model.add(Dropout(the_dropout))
                        model.add(BatchNormalization())
                        #全连接层
                        for i in range(the_dense_layers):
                            model.add(Dense(the_dense_layers,activation='relu'))
                            model.add(Dropout(the_dropout))
                            
                        # model.add(Flatten()) 
                        
                        
                        model.add(Dense(2, activation='softmax'))
                        
                        #sgd = SGD(learning_rate=0.01, momentum=0.9 , decay=0.1, nesterov=False)
                        
                        # learning_rate = 0.1
                        # decay = 0.001
                        # epochs = 50
                        # batch_size = 64
                        
                        
                        model.compile(  loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                                        optimizer=the_optimizer, metrics=['categorical_accuracy'])
                        print(model.summary())

                        epochs = 100
                        batch_size = the_batch_size
                        if(the_optimizer == sgd):
                            the_optimizer = 'sgd'
                        filepath = './models/{categorical_accuracy:.4f}_{epoch:02d}_'+f'dropout_{the_dropout}_batch_size_{the_batch_size}_optimizer_{the_optimizer}_dense_layers_{the_dense_layers}_lstm_layers_{the_lstm_layers}_unit_{the_units}.h5'
                        checkpoint = ModelCheckpoint(
                                            filepath=filepath,
                                            save_weights_only=False,
                                            monitor='categorical_accuracy',
                                            mode='max',
                                            save_best_only=True)
                        history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1,
                                            #callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
                                            callbacks=checkpoint)

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('./models/0.8597_99_dropout_0.05_batch_size_32_optimizer_adam_dense_layers_1_lstm_layers_1_unit_16.h5')

dataset = pd.read_csv('test.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']

lis = []
for i in range(28):
    lis.append(27-i)

Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
# lisindex = []
# for i in range(len(dataset.columns)):
#     lisindex.append(i)
# dataset = dataset.reindex(lisindex)
colIndex = ['Area','Minor_Axis', 'Eccentricity',
            'Eqdiasq','Convex_Area', 'Aspect_Ratio',
            'Compactness', 'Shapefactor_1', 
            'Shapefactor_3']
#colIndex = FReIndex(colIndex,lis)
dataset = pd.DataFrame(dataset,columns=colIndex)

X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#进行预测 make a prediction
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.9, random_state=random.randint(10,100))
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
# print ('train_x.shape, train_y.shape, test_x.shape, test_y.shape')
# print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
yhat = best_model.predict(X_test)
# print(yhat.shape)
print(yhat)
print(Y_test)
sorce = 0
for i in range(len(Y_test)):
    sorce += roc_auc_score(Y_test[i], yhat[i])

print('ACC:',sorce/len(Y_test))
#拆分训练集和测试集
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=random.randint(10,100))
# X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])